## Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## 1. Introduction
With the development of the global economy, more and more people change their places of residence due to the needs of study and work. In this project, we will take one of our clients as an example to discuss how to apply data analysis to find a satisfactory residence.

In this case, suppose our client currently lives in the downdown area of  Toronto, Ontario, Canada. He recently accepted a job offer in Manhattan. When he is satisfied with his current neighborhood, he is looking for similar neighborhoods in Manhattan, where the venues and living environment are similar. We will use data science techniques to help clients identify the most identical neighborhoods in Manhattan to his current residential area.

This project is not only suitable for our customers, but also for customers/readers who are interested in finding similar neighborhoods around the world.

## 2. Data Description
The analysis will be conducted into 3 parts:

##### Part 1: Collecting the Venues data of the Client 's Current Neighborhood in Toronto
We will first utilize the `Foursquare API` to collect the venues data of downtown Toronto. Then, the data will be pre-processing and convert into a clean dataframe that contains frequency of venues in the Downtown area of Toronto. 

##### Part 2: Explore the Neighborhoods in Manhattan, New York City
The Manhattan Dataset can be downloaded from https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json. We will explore Manhattan Neighborhoods by:
1. Collecting manhattan venues information using `Foursquare API`
2. Appling `Folium` to map out the neighborhoods in Manhattan
3. Cleanup the dataframe which matches the downtown Toronto dataframe.

##### Part 3: Compare the Neighborhood Clusters of Toronto and NYC
Finally, we will combine the dataframes and then apply `cluster analysis`. We will compare the clusters of Manhattan with the neighborhood of Toronto and find out the most similar neighborhoods in Manhattan to the client's current residential neighborhoods in Toronto.

## 3. Data Preparation and Exploration

### 3.1 Explore the Client 's Current Neighborhood in Toronto

In [70]:
import requests
import pandas
import numpy as np
import random

!pip install geopy
from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

! pip install folium==0.5.0
import folium # plotting library

print('Libraires imported')

Libraires imported


### 3.1.1 Explore the venues around the client home address.

In [71]:
#Foursquare ID, Secret and Access Token
CLIENT_ID = 'PXWYF4MCL5GNSRXXU1XNOKMBZUFZMSYSWAITELUQAQIKXFBD' 
CLIENT_SECRET = 'Z3YEX5LPJN3LTNCYSHIJBAS0RKN11QYVIL33IBBFL2NXLP2J'
VERSION = '20210401' 
LIMIT = 500 # A default Foursquare API limit value

The client is currently living in the Downtown Toronto, Toronto, Ontario, Canada. Let's convert his home address to latitude and longtide coordinates. 

In [72]:
CA_address='Downtown Toronto, Toronto,Canada'

geolocator=Nominatim(user_agent='foursquare_agent')
location=geolocator.geocode(CA_address)
latitude=location.latitude
longitude=location.longitude
print(latitude,longitude)

43.6563221 -79.3809161


In [73]:
import requests

radius=3000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results
#'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

{'meta': {'code': 200, 'requestId': '608043901048e00d0d040ecb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 236,
  'suggestedBounds': {'ne': {'lat': 43.683322127000025,
    'lng': -79.3436668217293},
   'sw': {'lat': 43.62932207299997, 'lng': -79.41816537827069}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a81ae339deb7d369fa7f146',
       'name': 'Hailed Coffee',
       'location': {'address': '44 Gerrard St W',
        'crossStreet': 'Yonge St',
        'lat': 43.65883296982352,
        'lng': -79.38368351986598,
        'labeledLatLngs': [{'label': 'display',
         

In [74]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [75]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pandas.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Hailed Coffee,Coffee Shop,43.658833,-79.383684
1,Banh Mi Boys,Sandwich Place,43.659292,-79.381949
2,Elgin And Winter Garden Theatres,Theater,43.653394,-79.378507
3,Blaze Pizza,Pizza Place,43.656518,-79.380015
4,Downtown Toronto,Neighborhood,43.653232,-79.385296


### 3.1.2 Pre-processing the Downtown Toronto Venues data

In [76]:
# calculate the frequency of venues
nearby_venues=nearby_venues.groupby('categories').count().rename(columns={'name':'count'}).sort_values('count',ascending=False)
nearby_venues['sum']=nearby_venues['count'].sum()
nearby_venues['Downtown Toronto']=nearby_venues['count']/nearby_venues['sum']

# Drop the extra columns
nearby_venues=nearby_venues.drop(['lat','lng','sum','count'],axis=1)

# reset the index name to Neighborhood
nearby_venues.index.names = ['Neighborhood']

# transpose the dataframe
nearby_venues1=nearby_venues.T
nearby_venues1

Neighborhood,Coffee Shop,Park,Café,Farmers Market,Gastropub,Japanese Restaurant,Beer Bar,Plaza,Thai Restaurant,Hotel,Sandwich Place,American Restaurant,Pizza Place,Furniture / Home Store,Historic Site,Neighborhood,Bookstore,Bakery,Ramen Restaurant,Organic Grocery,Salad Place,Record Shop,Restaurant,Monument / Landmark,Shopping Mall,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Street Art,Theater,Vegetarian / Vegan Restaurant,Museum,Hostel,Mediterranean Restaurant,Dance Studio,Arts & Crafts Store,Botanical Garden,Brewery,Bubble Tea Shop,Cheese Shop,Concert Hall,Cosmetics Shop,Creperie,Dessert Shop,Liquor Store,Diner,Food Truck,French Restaurant,Gourmet Shop,Grocery Store,Gym,Art Gallery,Italian Restaurant,Vietnamese Restaurant
Downtown Toronto,0.13,0.06,0.05,0.04,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


In [77]:
# passing the dataframe to DowntownToronto
DowntownToronto=nearby_venues1

#DowntownToronto=DowntownToronto.reset_index()
DowntownToronto

Neighborhood,Coffee Shop,Park,Café,Farmers Market,Gastropub,Japanese Restaurant,Beer Bar,Plaza,Thai Restaurant,Hotel,Sandwich Place,American Restaurant,Pizza Place,Furniture / Home Store,Historic Site,Neighborhood,Bookstore,Bakery,Ramen Restaurant,Organic Grocery,Salad Place,Record Shop,Restaurant,Monument / Landmark,Shopping Mall,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Street Art,Theater,Vegetarian / Vegan Restaurant,Museum,Hostel,Mediterranean Restaurant,Dance Studio,Arts & Crafts Store,Botanical Garden,Brewery,Bubble Tea Shop,Cheese Shop,Concert Hall,Cosmetics Shop,Creperie,Dessert Shop,Liquor Store,Diner,Food Truck,French Restaurant,Gourmet Shop,Grocery Store,Gym,Art Gallery,Italian Restaurant,Vietnamese Restaurant
Downtown Toronto,0.13,0.06,0.05,0.04,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


Now we finish collecting the venue data of downtown toronto. We will go ahead and collect the venue data of the Neighborhoods in Manhattan, New York City.

## 3.2 Explore the Neighborhoods in Manhattan, New York City


In [78]:
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import json
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium

from bs4 import BeautifulSoup

print('Libraries imported')

Libraries imported


### 3.2.1 Download NYC neighborhoods Data and then Retieve Manhattan data

In [79]:
#download Newyork data
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

with open('newyork_data.json') as json_data:
    newyork_data=json.load(json_data)
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [80]:
#abstract NYC neighborhood data 
ny_neighborhoods_data=newyork_data['features']


#create an empty dataframe
column_names=['Borough','Neighborhood','Latitude','Longitude']
ny_neighborhoods=pd.DataFrame(columns=column_names)

#loop through the data and fill the dataframe one row at a time
for data in ny_neighborhoods_data:
    borough=ny_neighborhoods_name=data['properties']['borough']
    ny_neighborhoods_name=data['properties']['name']
    
    ny_neighborhoods_latlon=data['geometry']['coordinates']
    ny_neighborhoods_lat=ny_neighborhoods_latlon[1]
    ny_neighborhoods_lon=ny_neighborhoods_latlon[0]
    
    ny_neighborhoods=ny_neighborhoods.append({'Borough':borough,
                                          'Neighborhood':ny_neighborhoods_name,
                                          'Latitude':ny_neighborhoods_lat,
                                          'Longitude':ny_neighborhoods_lon},ignore_index=True)
#Examine the df
ny_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [81]:
# Retreive manhattan_data
manhattan_data=ny_neighborhoods[ny_neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


### 3.2.2 Map Manhattan, NYC

In [82]:
# generate the map of NYC 
address='Manhattan, NY'

geolocator=Nominatim(user_agent='ny_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude

In [83]:
# Create a map of NY with neighborhoods superimposed on top

map_manhattan=folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(manhattan_data['Latitude'],manhattan_data['Longitude'],
                                       manhattan_data['Borough'],manhattan_data['Neighborhood']):
    label='{},{}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=1,popup=label,color='blue',
                       fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(map_manhattan)
map_manhattan

### 3.2.3 Explore Mahattan Neighborhoods

#### Define Foursquare Credentials and version

In [84]:
CLIENT_ID = 'PXWYF4MCL5GNSRXXU1XNOKMBZUFZMSYSWAITELUQAQIKXFBD' 
CLIENT_SECRET = 'Z3YEX5LPJN3LTNCYSHIJBAS0RKN11QYVIL33IBBFL2NXLP2J'
VERSION = '20210401' 
LIMIT = 100 # A default Foursquare API limit value

#### Create a function to repeat the same process to all the neighborhoods in Manhattan

In [85]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a new dataframe that contains manhattan vanues information

In [86]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [87]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3239, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [88]:
manhattan_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 328 uniques categories.


### 3.2.4 Pre-processing the Manhattan Venues data
####  Group the venues  by neighborhoods and calculated the frequency of venue categories

In [89]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup 

In [90]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup 

In [91]:
manhattan_grouped=manhattan_grouped.set_index('Neighborhood')
manhattan_grouped.head()


,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,South A

Now we finish collecting the venue data of the Neighborhoods in Manhattan, New York City. In the following, we will combining the dataframes of Downtown Toronton and Manhattan, and then conduct Clustering Analysis. 

## 4. Cluster Neighborhoods

### 4.1 Combine the dataframe
Cluster is a group of objects that are similar to other objects in the cluster and dissimilar to data points to other clusters. In the combined dataframe, we will get the similar neighborhoods in the cluster group where the Downtown Toronto neighborhood fell in.

In [92]:
DowntownToronto

Neighborhood,Coffee Shop,Park,Café,Farmers Market,Gastropub,Japanese Restaurant,Beer Bar,Plaza,Thai Restaurant,Hotel,Sandwich Place,American Restaurant,Pizza Place,Furniture / Home Store,Historic Site,Neighborhood,Bookstore,Bakery,Ramen Restaurant,Organic Grocery,Salad Place,Record Shop,Restaurant,Monument / Landmark,Shopping Mall,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Street Art,Theater,Vegetarian / Vegan Restaurant,Museum,Hostel,Mediterranean Restaurant,Dance Studio,Arts & Crafts Store,Botanical Garden,Brewery,Bubble Tea Shop,Cheese Shop,Concert Hall,Cosmetics Shop,Creperie,Dessert Shop,Liquor Store,Diner,Food Truck,French Restaurant,Gourmet Shop,Grocery Store,Gym,Art Gallery,Italian Restaurant,Vietnamese Restaurant
Downtown Toronto,0.13,0.06,0.05,0.04,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


In [93]:
manhattan_grouped.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,South A

In [94]:
#Combine data sets
df=pandas.concat([manhattan_grouped,DowntownToronto])

#drop the columuns where Null values is.
df=df.dropna(axis='columns')
df.tail()

,American Restaurant,Art Gallery,Arts & Crafts Store,Bakery,Beer Bar,Bookstore,Bubble Tea Shop,Café,Cheese Shop,Coffee Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Dessert Shop,Diner,Farmers Market,Food Truck,French Restaurant,Furniture / Home Store,Gastropub,Gourmet Shop,Grocery Store,Gym,Historic Site,Hostel,Hotel,Italian Restaurant,Japanese Restaurant,Liquor Store,Mediterranean Restaurant,Monument / Landmark,Museum,Organic Grocery,Park,Pizza Place,Plaza,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Sandwich Place,Shopping Mall,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Street Art,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
Upper West Side,0.020833,0.00,0.010417,0.041667,0.00,0.010417,0.00,0.031250,0.00,0.031250,0.00,0.00,0.010417,0.00,0.020833,0.010417,0.00,0.00,0.020833,0.00,0.00,0.00,0.000000,0.010417,0.00,0.00,0.010417,0.052083,0.010417,0.000000,0.03125,0.00,0.010417,0.00,0.000000,0.020833,0.000000,0.010417,0.00,0.020833,0.010417,0.000000,0.00,0.000000,0.00,0.00,0.010417,0.020833,0.00,0.020833,0.010417
Washington Heights,0.012048,0.00,0.000000,0.048193,0.00,0.000000,0.00,0.072289,0.00,0.024096,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.048193,0.024096,0.00,0.00,0.000000,0.012048,0.000000,0.012048,0.00000,0.00,0.000000,0.00,0.024096,0.036145,0.012048,0.012048,0.00,0.012048,0.000000,0.024096,0.00,0.024096,0.00,0.00,0.000000,0.012048,0.00,0.000000,0.000000
West Village,0.050000,0.01,0.000000,0.020000,0.01,0.010000,0.00,0.010000,0.00,0.030000,0.00,0.03,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.020000,0.00,0.02,0.01,0.000000,0.010000,0.00,0.00,0.000000,0.110000,0.000000,0.000000,0.01000,0.00,0.000000,0.00,0.040000,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.03,0.000000,0.000000
Yorkville,0.000000,0.00,0.000000,0.010000,0.00,0.010000,0.00,0.010000,0.00,0.050000,0.00,0.01,0.000000,0.00,0.010000,0.030000,0.01,0.00,0.010000,0.00,0.00,0.01,0.000000,0.060000,0.00,0.00,0.000000,0.080000,0.030000,0.010000,0.01000,0.01,0.000000,0.00,0.020000,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.000000,0.020000
Downtown Toronto,0.020000,0.01,0.010000,0.020000,0.03,0.020000,0.01,0.050000,0.01,0.130000,0.01,0.01,0.010000,0.01,0.010000,0.010000,0.04,0.01,0.010000,0.02,0.04,0.01,0.010000,0.010000,0.02,0.01,0.030000,0.010000,0.030000,0.010000,0.01000,0.01,0.010000,0.01,0.060000,0.020000,0.030000,0.010000,0.01,0.010000,0.010000,0.030000,0.01,0.010000,0.01,0.01,0.010000,0.030000,0.01,0.010000,0.010000


### 4.2 Cluster Analysis

In [95]:
from sklearn.preprocessing import StandardScaler
X=df.values[:,1:]
X=np.nan_to_num(X) #replace Null to Zero
Clus_dataset=StandardScaler().fit_transform(X)
Clus_dataset

array([[-0.66383685, -0.37701046, -1.13382755, ..., -0.4274998 ,
        -0.60681678, -0.70553183],
       [-0.66383685, -0.37701046, -0.07831402, ..., -0.4274998 ,
         0.77679   ,  0.54624264],
       [ 2.88421336, -0.37701046, -1.13382755, ..., -0.4274998 ,
        -0.60681678, -0.70553183],
       ...,
       [ 0.13447445, -0.37701046, -0.15219996, ...,  1.79944594,
        -0.60681678, -0.70553183],
       [-0.66383685, -0.37701046, -0.64301375, ..., -0.4274998 ,
        -0.60681678,  1.62276868],
       [ 0.13447445,  1.42791537, -0.15219996, ...,  0.31481544,
         0.67993752,  0.45861843]])

In [96]:
clusterNum=6
k_means=KMeans(init='k-means++',n_clusters=clusterNum,n_init=12)
k_means.fit(X)
labels=k_means.labels_
print(labels)

[3 4 1 1 1 4 2 0 1 4 1 1 1 0 1 0 1 2 1 0 1 1 5 2 1 4 1 1 3 1 3 1 1 0 1 1 1
 0 1 1 4]


In [97]:
#add the labels to the dataframe
df['Cluster']=labels
df.tail()

,American Restaurant,Art Gallery,Arts & Crafts Store,Bakery,Beer Bar,Bookstore,Bubble Tea Shop,Café,Cheese Shop,Coffee Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Dessert Shop,Diner,Farmers Market,Food Truck,French Restaurant,Furniture / Home Store,Gastropub,Gourmet Shop,Grocery Store,Gym,Historic Site,Hostel,Hotel,Italian Restaurant,Japanese Restaurant,Liquor Store,Mediterranean Restaurant,Monument / Landmark,Museum,Organic Grocery,Park,Pizza Place,Plaza,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Sandwich Place,Shopping Mall,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Street Art,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Cluster
Upper West Side,0.020833,0.00,0.010417,0.041667,0.00,0.010417,0.00,0.031250,0.00,0.031250,0.00,0.00,0.010417,0.00,0.020833,0.010417,0.00,0.00,0.020833,0.00,0.00,0.00,0.000000,0.010417,0.00,0.00,0.010417,0.052083,0.010417,0.000000,0.03125,0.00,0.010417,0.00,0.000000,0.020833,0.000000,0.010417,0.00,0.020833,0.010417,0.000000,0.00,0.000000,0.00,0.00,0.010417,0.020833,0.00,0.020833,0.010417,1
Washington Heights,0.012048,0.00,0.000000,0.048193,0.00,0.000000,0.00,0.072289,0.00,0.024096,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.048193,0.024096,0.00,0.00,0.000000,0.012048,0.000000,0.012048,0.00000,0.00,0.000000,0.00,0.024096,0.036145,0.012048,0.012048,0.00,0.012048,0.000000,0.024096,0.00,0.024096,0.00,0.00,0.000000,0.012048,0.00,0.000000,0.000000,0
West Village,0.050000,0.01,0.000000,0.020000,0.01,0.010000,0.00,0.010000,0.00,0.030000,0.00,0.03,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.020000,0.00,0.02,0.01,0.000000,0.010000,0.00,0.00,0.000000,0.110000,0.000000,0.000000,0.01000,0.00,0.000000,0.00,0.040000,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.03,0.000000,0.000000,1
Yorkville,0.000000,0.00,0.000000,0.010000,0.00,0.010000,0.00,0.010000,0.00,0.050000,0.00,0.01,0.000000,0.00,0.010000,0.030000,0.01,0.00,0.010000,0.00,0.00,0.01,0.000000,0.060000,0.00,0.00,0.000000,0.080000,0.030000,0.010000,0.01000,0.01,0.000000,0.00,0.020000,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.000000,0.020000,1
Downtown Toronto,0.020000,0.01,0.010000,0.020000,0.03,0.020000,0.01,0.050000,0.01,0.130000,0.01,0.01,0.010000,0.01,0.010000,0.010000,0.04,0.01,0.010000,0.02,0.04,0.01,0.010000,0.010000,0.02,0.01,0.030000,0.010000,0.030000,0.010000,0.01000,0.01,0.010000,0.01,0.060000,0.020000,0.030000,0.010000,0.01,0.010000,0.010000,0.030000,0.01,0.010000,0.01,0.01,0.010000,0.030000,0.01,0.010000,0.010000,4


### 4.3 Convert the frequency to the orders of top venues

In [98]:
df_grouped=df.drop(['Cluster'],axis=1)
df_grouped=df_grouped.reset_index()
df_grouped=df_grouped.rename(columns={'index':'Neighborhood'})
df_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Bakery,Beer Bar,Bookstore,Bubble Tea Shop,Café,Cheese Shop,Coffee Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Dessert Shop,Diner,Farmers Market,Food Truck,French Restaurant,Furniture / Home Store,Gastropub,Gourmet Shop,Grocery Store,Gym,Historic Site,Hostel,Hotel,Italian Restaurant,Japanese Restaurant,Liquor Store,Mediterranean Restaurant,Monument / Landmark,Museum,Organic Grocery,Park,Pizza Place,Plaza,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Sandwich Place,Shopping Mall,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Street Art,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Battery Park City,0.011628,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.058140,0.0,0.011628,0.0,0.000000,0.011628,0.00,0.0,0.000000,0.000000,0.00,0.0,0.023256,0.011628,0.034884,0.00,0.0,0.046512,0.011628,0.011628,0.0,0.011628,0.011628,0.000000,0.00,0.093023,0.023256,0.023256,0.000000,0.00,0.000000,0.000000,0.023256,0.023256,0.00,0.0,0.011628,0.0,0.00,0.00,0.000000,0.000000
1,Carnegie Hill,0.010753,0.000000,0.0,0.021505,0.000000,0.032258,0.00,0.064516,0.00,0.086022,0.0,0.021505,0.0,0.010753,0.000000,0.00,0.0,0.010753,0.032258,0.00,0.0,0.010753,0.032258,0.032258,0.00,0.0,0.010753,0.021505,0.010753,0.0,0.010753,0.000000,0.010753,0.00,0.000000,0.032258,0.000000,0.010753,0.00,0.010753,0.010753,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.00,0.010753,0.010753
2,Central Harlem,0.044444,0.044444,0.0,0.000000,0.022222,0.022222,0.00,0.022222,0.00,0.000000,0.0,0.022222,0.0,0.000000,0.022222,0.00,0.0,0.000000,0.044444,0.00,0.0,0.000000,0.022222,0.022222,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.022222,0.022222,0.000000,0.000000,0.00,0.022222,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.00,0.000000,0.000000
3,Chelsea,0.040000,0.040000,0.0,0.050000,0.010000,0.020000,0.00,0.020000,0.01,0.070000,0.0,0.010000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.040000,0.01,0.0,0.000000,0.010000,0.010000,0.00,0.0,0.020000,0.030000,0.010000,0.0,0.000000,0.000000,0.000000,0.00,0.020000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.010000,0.0,0.02,0.01,0.000000,0.000000
4,Chinatown,0.040000,0.000000,0.0,0.080000,0.000000,0.000000,0.02,0.000000,0.00,0.020000,0.0,0.010000,0.0,0.000000,0.030000,0.01,0.0,0.000000,0.000000,0.01,0.0,0.000000,0.010000,0.000000,0.01,0.0,0.010000,0.010000,0.000000,0.0,0.000000,0.000000,0.010000,0.01,0.000000,0.010000,0.000000,0.010000,0.01,0.000000,0.000000,0.020000,0.000000,0.01,0.0,0.000000,0.0,0.01,0.00,0.010000,0.020000


In [99]:
num_top_venues = 5

for hood in df_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_grouped[df_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
          venue  freq
0          Park  0.09
1   Coffee Shop  0.06
2         Hotel  0.05
3           Gym  0.03
4  Gourmet Shop  0.02


----Carnegie Hill----
               venue  freq
0        Coffee Shop  0.09
1               Café  0.06
2        Pizza Place  0.03
3  French Restaurant  0.03
4                Gym  0.03


----Central Harlem----
                 venue  freq
0  American Restaurant  0.04
1          Art Gallery  0.04
2    French Restaurant  0.04
3         Dessert Shop  0.02
4        Grocery Store  0.02


----Chelsea----
                 venue  freq
0          Coffee Shop  0.07
1               Bakery  0.05
2  American Restaurant  0.04
3          Art Gallery  0.04
4    French Restaurant  0.04


----Chinatown----
                 venue  freq
0               Bakery  0.08
1  American Restaurant  0.04
2         Dessert Shop  0.03
3      Bubble Tea Shop  0.02
4       Sandwich Place  0.02


----Civic Center----
                 venue  freq
0          Coffee S

In [100]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [101]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Gourmet Shop,Shopping Mall,Sandwich Place,Plaza,Pizza Place,Cosmetics Shop
1,Carnegie Hill,Coffee Shop,Café,French Restaurant,Gym,Bookstore,Pizza Place,Grocery Store,Bakery,Cosmetics Shop,Italian Restaurant
2,Central Harlem,American Restaurant,Art Gallery,French Restaurant,Restaurant,Dessert Shop,Park,Pizza Place,Gym,Cosmetics Shop,Grocery Store
3,Chelsea,Coffee Shop,Bakery,American Restaurant,Art Gallery,French Restaurant,Italian Restaurant,Park,Thai Restaurant,Bookstore,Café
4,Chinatown,Bakery,American Restaurant,Dessert Shop,Sandwich Place,Bubble Tea Shop,Coffee Shop,Vietnamese Restaurant,Spanish Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant
5,Civic Center,Coffee Shop,American Restaurant,French Restaurant,Hotel,Italian Restaurant,Park,Sandwich Place,Gym,Bakery,Dance Studio
6,Clinton,Theater,American Restaurant,Gym,Italian Restaurant,Coffee Shop,Sandwich Place,Hotel,Pizza Place,Mediterranean Restaurant,Park
7,East Harlem,Thai Restaurant,Bakery,Sandwich Place,Park,Pizza Place,Café,Grocery Store,Gym,Historic Site,Dance Studio
8,East Village,Pizza Place,Coffee Shop,Italian Restaurant,Vegetarian / Vegan Restaurant,Dessert Shop,Ramen Restaurant,Vietnamese Restaurant,Japanese Restaurant,Cheese Shop,Organic Grocery
9,Financial District,Coffee Shop,Pizza Place,American Restaurant,Italian Restaurant,Gym,Hotel,Monument / Landmark,Park,Café,Sandwich Place


In [102]:
# add clustering labels
neighborhoods_venues_sorted['Cluster']=labels
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Gourmet Shop,Shopping Mall,Sandwich Place,Plaza,Pizza Place,Cosmetics Shop,3
1,Carnegie Hill,Coffee Shop,Café,French Restaurant,Gym,Bookstore,Pizza Place,Grocery Store,Bakery,Cosmetics Shop,Italian Restaurant,4
2,Central Harlem,American Restaurant,Art Gallery,French Restaurant,Restaurant,Dessert Shop,Park,Pizza Place,Gym,Cosmetics Shop,Grocery Store,1
3,Chelsea,Coffee Shop,Bakery,American Restaurant,Art Gallery,French Restaurant,Italian Restaurant,Park,Thai Restaurant,Bookstore,Café,1
4,Chinatown,Bakery,American Restaurant,Dessert Shop,Sandwich Place,Bubble Tea Shop,Coffee Shop,Vietnamese Restaurant,Spanish Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,1
5,Civic Center,Coffee Shop,American Restaurant,French Restaurant,Hotel,Italian Restaurant,Park,Sandwich Place,Gym,Bakery,Dance Studio,4
6,Clinton,Theater,American Restaurant,Gym,Italian Restaurant,Coffee Shop,Sandwich Place,Hotel,Pizza Place,Mediterranean Restaurant,Park,2
7,East Harlem,Thai Restaurant,Bakery,Sandwich Place,Park,Pizza Place,Café,Grocery Store,Gym,Historic Site,Dance Studio,0
8,East Village,Pizza Place,Coffee Shop,Italian Restaurant,Vegetarian / Vegan Restaurant,Dessert Shop,Ramen Restaurant,Vietnamese Restaurant,Japanese Restaurant,Cheese Shop,Organic Grocery,1
9,Financial District,Coffee Shop,Pizza Place,American Restaurant,Italian Restaurant,Gym,Hotel,Monument / Landmark,Park,Café,Sandwich Place,4


In [103]:
manhattan_merged=manhattan_data
manhattan_merged=manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')
manhattan_merged.tail()

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
35,Manhattan,Turtle Bay,40.752042,-73.967708,Italian Restaurant,Coffee Shop,Park,Japanese Restaurant,Ramen Restaurant,Café,Diner,Hotel,Plaza,French Restaurant,1
36,Manhattan,Tudor City,40.746917,-73.971219,Park,Café,Coffee Shop,Vietnamese Restaurant,Pizza Place,Thai Restaurant,Diner,Gym,Restaurant,French Restaurant,0
37,Manhattan,Stuyvesant Town,40.731000,-73.974052,Park,Coffee Shop,Farmers Market,Vietnamese Restaurant,Dance Studio,Grocery Store,Gourmet Shop,Gastropub,Furniture / Home Store,French Restaurant,3
38,Manhattan,Flatiron,40.739673,-73.990947,Italian Restaurant,American Restaurant,Japanese Restaurant,Coffee Shop,Mediterranean Restaurant,Dessert Shop,Furniture / Home Store,Sporting Goods Shop,Park,Café,1
39,Manhattan,Hudson Yards,40.756658,-74.000111,American Restaurant,Hotel,Café,Italian Restaurant,Thai Restaurant,Coffee Shop,Gym,Park,Restaurant,Art Gallery,1


In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(clusterNum)
ys = [i + x + (i*x)**2 for i in range(clusterNum)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Compare the Neighborhood Clusters of Toronto and NYC


In [111]:
manhattan_similar_neighborhoods=manhattan_merged.loc[manhattan_merged['Cluster']==4]
manhattan_similar_neighborhoods

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
26,Manhattan,Morningside Heights,40.808000,-73.963896,Coffee Shop,Bookstore,Park,American Restaurant,Café,Sandwich Place,Food Truck,Salad Place,Farmers Market,Arts & Crafts Store,4
29,Manhattan,Financial District,40.707107,-74.010665,Coffee Shop,Pizza Place,American Restaurant,Italian Restaurant,Gym,Hotel,Monument / Landmark,Park,Café,Sandwich Place,4
30,Manhattan,Carnegie Hill,40.782683,-73.953256,Coffee Shop,Café,French Restaurant,Gym,Bookstore,Pizza Place,Grocery Store,Bakery,Cosmetics Shop,Italian Restaurant,4
32,Manhattan,Civic Center,40.715229,-74.005415,Coffee Shop,American Restaurant,French Restaurant,Hotel,Italian Restaurant,Park,Sandwich Place,Gym,Bakery,Dance Studio,4


In [114]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
map_clusters

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_similar_neighborhoods['Latitude'], manhattan_similar_neighborhoods['Longitude'], manhattan_merged['Neighborhood'], manhattan_similar_neighborhoods['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [110]:
neighborhoods_venues_sorted[neighborhoods_venues_sorted['Cluster']==4]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
1,Carnegie Hill,Coffee Shop,Café,French Restaurant,Gym,Bookstore,Pizza Place,Grocery Store,Bakery,Cosmetics Shop,Italian Restaurant,4
5,Civic Center,Coffee Shop,American Restaurant,French Restaurant,Hotel,Italian Restaurant,Park,Sandwich Place,Gym,Bakery,Dance Studio,4
9,Financial District,Coffee Shop,Pizza Place,American Restaurant,Italian Restaurant,Gym,Hotel,Monument / Landmark,Park,Café,Sandwich Place,4
25,Morningside Heights,Coffee Shop,Bookstore,Park,American Restaurant,Café,Sandwich Place,Food Truck,Salad Place,Farmers Market,Arts & Crafts Store,4
40,Downtown Toronto,Coffee Shop,Park,Café,Gastropub,Farmers Market,Plaza,Thai Restaurant,Hotel,Beer Bar,Japanese Restaurant,4


Based on the cluster results, we found that there are three neighborhoods in Manhattan that are similar to the Downtown Toronto: **Carnegie Hill, Civic Center, Morningside Heights and Financial District**. Our client can focus on these three neighborhoods and pick out the best fit one based on his own perferences.